# Preface
The goal of these workshops is to do some exercices using the MAD-X models to help you to visualoze some transverse dynamics concepts.


# Tutorial 1: Part 1

The goal of this first exercice is to get familiar with the pyhton interface.

In [ ]:
from matplotlib import pyplot as plt

In [2]:
from cpymad.madx import Madx 
myMad = Madx()

In [3]:
myString='''
l_cell=100;
quadrupoleLenght=5;
f=200;
myK:=1/f/quadrupoleLenght;// m^-2

! *************  Definition of magnet  **************

QF: quadrupole, L=quadrupoleLenght, K1:=myK;
QD: quadrupole, L=quadrupoleLenght, K1:=-myK;

! ************** Definition of sequence *************

myCell:sequence, refer=entry, L=L_CELL;
quadrupole1: QF, at=0;
marker1: marker, at=25;
quadrupole2: QD, at=50;
endsequence;

! ********************* Definition of beam **********

beam, particle=proton, energy=2;

! ***************** Use of the sequence *************

use, sequence=myCell;

title, 'My first twiss';
twiss, file=./secondpart/firstTwiss.txt;

'''

In [4]:
myMad.input(myString);

In [6]:
list(myMad.table)

['summ', 'twiss']

In [9]:
myMad.table.twiss.betx

array([463.62328831, 463.62328831, 419.39486679, 419.39486679,
       369.77916163, 369.77916163, 463.62328831, 463.62328831])